In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 20610602
paper_name = 'cooper_fields_2010' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/SupplementalTable4.xlsx', sheet_name='SupplementalTable4.txt')

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 4382 x 19


In [7]:
original_data.head()

,Sysname,NAME,RPS19,arg1,gshbiotin,NacOrn,LeuIleCit,GlnVal,MetPro,Thr,Ala,Ser,Asn-Tyr,Gly,LysA,Orn,LysB,Glu,Asp
0,YBR208C,"DUR1,2",0,3.54333,-,2.42612,1.15477,1.15431,2.75038,0.724826,1.94145,0,0.764878,2.47268,1.63992,3.57424,1.41873,7.66189,8.59263
1,YGL234W,"ADE5,7",1.98892,1.52266,1.84747,0.362926,0.874521,1.2828,1.065,1.49768,1.20143,-,1.17493,1.15068,1.107,1.9984,0.959894,8.52946,0.716635
2,YAL043C-A,YAL043C-A,-,0.668344,0.30119,0.416194,0.640974,0.700286,0.729417,2.01743,0.821994,0.0693558,1.48113,0.714057,1.07648,0.117303,1.45502,1.46003,1.24339
3,YAL058C-A,YAL058C-A,-,0.780769,0.383783,0.264115,0.841378,1.5815,0.643997,1.32022,0.905065,2.92555,0.640073,0.309209,0.916185,1.23094,0.834674,2.19552,1.63379
4,YAL064C-A,YAL064C-A,-,1.03781,0.416667,0.330396,0.776574,1.14922,1.77936,0.0945348,1.35279,1.14611,0.742243,1.15692,0.651835,1.05495,0.52626,1.6135,2.27627


In [8]:
original_data['orf'] = original_data['Sysname'].astype(str)

In [9]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [12]:
original_data.loc[original_data['orf']=='YML048WA-','orf'] = 'YML048W-A'

In [13]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [14]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [Sysname, NAME, RPS19, arg1, gshbiotin, NacOrn, LeuIleCit, GlnVal, MetPro, Thr, Ala, Ser, Asn-Tyr, Gly, LysA, Orn, LysB, Glu, Asp, orf]
Index: []


In [16]:
original_data.drop(columns=['Sysname','NAME'], inplace=True)

In [17]:
original_data.set_index('orf', inplace=True)

In [18]:
original_data = original_data.apply(pd.to_numeric, axis=1, errors='coerce')

In [19]:
original_data = original_data.groupby(original_data.index).mean()

In [20]:
original_data['LysA'] = original_data[['LysA','LysB']].mean(axis=1)

In [21]:
original_data.drop(columns=['LysB'], inplace=True)

In [22]:
original_data.shape

(4312, 16)

# Prepare the final dataset

In [23]:
data = original_data.copy()

In [24]:
dataset_ids = [32, 730, 731, 732, 733, 734, 735, 736, 737, 738, 739, 740, 741, 742, 743, 744]
datasets = datasets.reindex(index=dataset_ids)

In [25]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [26]:
data.head()

dataset_id,32,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744
data_type,value,value,value,value,value,value,value,value,value,value,value,value,value,value,value,value
orf,,,,,,,,,,,,,,,,
YAL004W,NaN,1.355685,NaN,0.703685,1.172069,1.508307,0.901307,1.284380,1.808554,0.453896,1.361483,1.204610,0.677463,0.734968,0.902132,0.890400
YAL005C,NaN,0.532776,0.318279,0.133498,0.545810,0.653070,0.590081,0.428283,0.823910,0.263513,0.719861,0.515424,0.354860,0.058932,0.306538,0.368548
YAL007C,NaN,0.655952,0.262740,1.056912,0.857829,0.962162,1.031008,0.546406,1.110576,1.177857,0.855912,0.844396,0.369063,0.110178,0.403306,0.745529
YAL008W,NaN,0.700830,0.258468,0.393738,0.977303,0.944960,0.905943,0.916037,0.996158,0.439390,1.123170,0.882361,0.490268,0.544036,0.487901,0.911223
YAL009W,0.0,1.633923,0.267013,0.397799,1.095166,1.033210,1.084407,0.891140,0.876589,0.431066,1.186386,0.988467,5.397268,1.178647,1.310748,1.658009


## Subset to the genes currently in SGD

In [27]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 21


In [28]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,32,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744
,data_type,value,value,value,value,value,value,value,value,value,value,value,value,value,value,value,value
gene_id,orf,,,,,,,,,,,,,,,,
1863,YAL004W,NaN,1.355685,NaN,0.703685,1.172069,1.508307,0.901307,1.284380,1.808554,0.453896,1.361483,1.204610,0.677463,0.734968,0.902132,0.890400
4,YAL005C,NaN,0.532776,0.318279,0.133498,0.545810,0.653070,0.590081,0.428283,0.823910,0.263513,0.719861,0.515424,0.354860,0.058932,0.306538,0.368548
5,YAL007C,NaN,0.655952,0.262740,1.056912,0.857829,0.962162,1.031008,0.546406,1.110576,1.177857,0.855912,0.844396,0.369063,0.110178,0.403306,0.745529
6,YAL008W,NaN,0.700830,0.258468,0.393738,0.977303,0.944960,0.905943,0.916037,0.996158,0.439390,1.123170,0.882361,0.490268,0.544036,0.487901,0.911223
7,YAL009W,0.0,1.633923,0.267013,0.397799,1.095166,1.033210,1.084407,0.891140,0.876589,0.431066,1.186386,0.988467,5.397268,1.178647,1.310748,1.658009


# Normalize

In [29]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [30]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [31]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,32,730,731,732,733,734,735,736,737,738,...,735,736,737,738,739,740,741,742,743,744
,data_type,value,value,value,value,value,value,value,value,value,value,...,valuez,valuez,valuez,valuez,valuez,valuez,valuez,valuez,valuez,valuez
gene_id,orf,,,,,,,,,,,,,,,,,,,,,
1863,YAL004W,NaN,1.355685,NaN,0.703685,1.172069,1.508307,0.901307,1.284380,1.808554,0.453896,...,0.055295,0.465508,2.518488,-0.305186,1.188096,0.638910,-0.023663,0.003595,-0.143500,-0.097413
4,YAL005C,NaN,0.532776,0.318279,0.133498,0.545810,0.653070,0.590081,0.428283,0.823910,0.263513,...,-0.584633,-0.152084,-0.305131,-0.421808,-0.616402,-0.772837,-0.335805,-0.600442,-0.415641,-0.286158
5,YAL007C,NaN,0.655952,0.262740,1.056912,0.857829,0.962162,1.031008,0.546406,1.110576,1.177857,...,0.321979,-0.066869,0.516926,0.138289,-0.233774,-0.098963,-0.322063,-0.554654,-0.371425,-0.149810
6,YAL008W,NaN,0.700830,0.258468,0.393738,0.977303,0.944960,0.905943,0.916037,0.996158,0.439390,...,0.064828,0.199784,0.188815,-0.314072,0.517864,-0.021193,-0.204788,-0.167003,-0.332772,-0.089881
7,YAL009W,0.0,1.633923,0.267013,0.397799,1.095166,1.033210,1.084407,0.891140,0.876589,0.431066,...,0.431777,0.181823,-0.154067,-0.319171,0.695651,0.196157,4.543108,0.400020,0.043206,0.180219


# Print out

In [32]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [33]:
from IO.save_data_to_db3 import *

In [34]:
save_data_to_db(data_all, paper_pmid)

Deleting all datasets for PMID 20610602...


  0%|          | 0/16 [00:00<?, ?it/s]

Inserting the new data...


100%|██████████| 16/16 [01:45<00:00,  6.60s/it]

Updating the data_modified_on field...
